# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# enter your code here
orders = pd.read_csv("../../lab-subsetting-and-descriptive-stats/your-code/Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# your code here

customers_spent = orders.groupby(['CustomerID'])['amount_spent', 'Quantity'].agg('sum')
customers_spent.head()

,amount_spent,Quantity
CustomerID,,
12346,77183.60,74215
12347,4310.00,2458
12348,1797.24,2341
12349,1757.55,631
12350,334.40,197


In [4]:
def quantile_spent(x):
    return customers_spent[customers_spent['amount_spent'] > customers_spent['amount_spent'].quantile(x)]['amount_spent']

quantile_spent(0.90).head()

CustomerID
12346    77183.60
12347     4310.00
12357     6207.67
12359     6372.58
12362     5226.23
Name: amount_spent, dtype: float64

In [6]:
P90 = customers_spent['amount_spent'].quantile(0.90)
P75 = customers_spent['amount_spent'].quantile(0.75)

def customer_label(x):
    if x >= P90:
        return 'VIP'
    elif x >= P75:
        return 'Preferred Customer'
    else:
        return 'Regular'
    
customers_spent['Customer Type'] = customers_spent['amount_spent'].apply(customer_label)

In [7]:
customers_spent.head()

,amount_spent,Quantity,Customer Type
CustomerID,,,
12346,77183.60,74215,VIP
12347,4310.00,2458,VIP
12348,1797.24,2341,Preferred Customer
12349,1757.55,631,Preferred Customer
12350,334.40,197,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [8]:
# your code here
orders = orders.join(customers_spent['Customer Type'], on = 'CustomerID')

In [9]:
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,Customer Type
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,VIP
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,VIP
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP


In [20]:
count_vips = orders.groupby(['Country', 'Customer Type']).agg('count')

In [26]:
count_vips.sort_values(by='Customer Type', ascending = False)

,,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,amount_spent
Country,Customer Type,,,,,,,,,,,,,
Finland,VIP,331,331,331,331,331,331,331,331,331,331,331,331,331
Denmark,VIP,112,112,112,112,112,112,112,112,112,112,112,112,112
Portugal,VIP,681,681,681,681,681,681,681,681,681,681,681,681,681
Israel,VIP,171,171,171,171,171,171,171,171,171,171,171,171,171
Cyprus,VIP,248,248,248,248,248,248,248,248,248,248,248,248,248
EIRE,VIP,7238,7238,7238,7238,7238,7238,7238,7238,7238,7238,7238,7238,7238
Singapore,VIP,222,222,222,222,222,222,222,222,222,222,222,222,222
Channel Islands,VIP,364,364,364,364,364,364,364,364,364,364,364,364,364
Poland,VIP,149,149,149,149,149,149,149,149,149,149,149,149,149


In [29]:
countervips = orders[orders['Customer Type']=='VIP'].groupby(['Country'])['Customer Type'].agg('count')

In [34]:
countervips.sort_values(ascending = False).head(1)

Country
United Kingdom    131497
Name: Customer Type, dtype: int64

In [35]:
counterpreferred = orders[orders['Customer Type']=='Preferred Customer'].groupby(['Country'])['Customer Type'].agg('count')

In [36]:
counterpreferred.sort_values(ascending = False).head(1)

Country
United Kingdom    90138
Name: Customer Type, dtype: int64

In [37]:
count = counterpreferred + countervips
count.sort_values(ascending=False).head(1)

Country
United Kingdom    221635.0
Name: Customer Type, dtype: float64